In [ ]:
!pip3 install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 4.2 MB 30.9 MB/s 
     |████████████████████████████████| 1.2 MB 49.2 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 51.7 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=26d1b11254b065a6e91a4841e68815204dcae3355ef3dffd1efb3e1d859a7800
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/myColab/AlgoTrader

Mounted at /content/drive
/content/drive/MyDrive/myColab/AlgoTrader


In [ ]:
import pandas as pd
import string
import re
fileName = "Glue/CoLA/dev.tsv"
dataset = []
punctuation_string = string.punctuation
#print(punctuation_string)
with open(fileName, 'r') as fin:
  lines = fin.readlines()
  for line in lines:
    #print(line)
    line = line.strip().split('\t')
    line = line[3]
    line = re.sub('[{}]'.format(punctuation_string),"",line)
    #line.translate(string.punctuation)
    #line = line.split(' ')
    #tmp = []
    #tmp.append(line)
    dataset.append(line)

In [ ]:
import pandas as pd
import string
import re
punctuation_string = string.punctuation
filename = 'NYTimes_all_headline.txt'
dataset = []
with open(filename, 'r') as fin:
  lines = fin.readlines()
  for line in lines:
    line = line.strip()
    line = re.sub('[{}]'.format(punctuation_string),"",line)
    dataset.append(line)

In [ ]:
print(dataset)

['The sailors rode the breeze clear of the rocks', 'The weights made the rope stretch over the pulley', 'The mechanical doll wriggled itself loose', 'If you had eaten more you would want less', 'As you eat the most you want the least', 'The more you would want the less you would eat', 'I demand that the more John eat the more he pays', 'Mary listens to the Grateful Dead she gets depressed', 'The angrier Mary got the more she looked at pictures', 'The higher the stakes the lower his expectations are', 'The more Fred is obnoxious the less attention you should pay to him', 'John was lots more obnoxious than Fred', 'The more people you give beer to the more people get sick', 'The more does Bill smoke the more Susan hates him', 'The more pictures of him that appear in the news the more embarrassed John becomes', 'Every senator seems to become more corrupt as he talks to more lobbyists', 'Who does John visit Sally because he likes', 'Marianne did not leave', 'He could not have been working',

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
#train_sentences = ["Your set of sentences",
#                   "Model will automatically add the noise", 
#                   "And re-construct it",
#                   "You should provide at least 1k sentences"]
train_sentences = dataset
# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
train_batch_size = 8
# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
dev_samples = []
test_samples = []
# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)
#dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')
#test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
# Call the fit method
#print(train_loss)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    #evaluator=dev_evaluator,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('sbert/tsdae-model')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/131 [00:00<?, ?it/s]

In [ ]:
# Two lists of sentences
sentences1 = ['The sailors rode the breeze clear of the rocks',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The sailors rode the breeze clear of the roses',
              'A woman is watching TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
print(embeddings1.shape)
#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)\

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

torch.Size([1, 768])
The sailors rode the breeze clear of the rocks 		 The sailors rode the breeze clear of the roses 		 Score: 0.9750


In [ ]:
s = ''
for i in range(513):
  s += 'test'
  s+= ' '
l = [s]
print(l)
embeddings_test = model.encode(l, convert_to_tensor=True)
print(embeddings_test)

['test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test tes

In [ ]:
print(model.max_seq_length)

512
